In [93]:
# -*-coding:utf-8-*-
import pymysql
import os
import xlrd
# import pretty_errors

# 序时账里只取买入的数据

# connection = pymysql.connect(host='39.105.9.20', user='root', passwd='bigdata_oil',
#                 db='cxd_data', port=3306, charset='utf8')

connection = pymysql.connect(host='39.105.9.20', user='root', passwd='bigdata_oil',
                db='cxd_test', port=3306, charset='utf8')

cursor = connection.cursor()

oil_key = ['油', '沥青', '92', '95', '0', '35', '10']
trade_key = ['购', '销']

docxlist = {}
com_name = ''

def name_filter(com_name): # 有些特殊的记账方式对应的名字
    filter = ['单位', '往来单位', '应付账款', '客户']
    com = com_name
    for f in filter:
        if com_name.startswith(f):
            com = com_name[len(f):]
    return com


def gci(filepath):
    # 遍历filepath下所有文件，包括子目录
    global com_name
    files = os.listdir(filepath)
    for fi in files:
        if fi == '.DS_Store':
            continue
        fi_d = os.path.join(filepath, fi)
        if os.path.isdir(fi_d):
            if '公司' in fi:
                com_name = fi
                docxlist[com_name] = []
            gci(fi_d)
        else:
            docxlist[com_name].append(fi_d)
    return docxlist


def read_xls(path):
    # xls文件解析
    record = []
    data = None
    try:
        data = xlrd.open_workbook(path, encoding_override="cp936")  # 打开xls文件
    except Exception as e:
        print(e)
        data = xlrd.open_workbook(path, encoding_override="cp1252")
    finally:
        if data.sheets():
            table = data.sheets()[0]  # 打开第一张表
            nrows = table.nrows  # 获取表的行数
            for i in range(nrows):  # 循环逐行打印
                if i == 0:  # 跳过第一行
                    continue
                record.append(table.row_values(i))
        return record


def data_merge(com, original_data, path):
    # 将序时账合并为对应的一条数据
    # 样例数据
    # [['10', '*', '', '', '2019-10-31', '记-1', '购货', '140587', '库存商品 - 0号  车用柴油(IV)', '', '', '', '', '',
    # 'RMB', '', 199821.24, 199821.24, 0.0, 'Manager', '', 'Manager', '', '', '', '32.221', '6,201.58'],
    # ['10', '*', '', '', '', '', '购货', '140564', '库存商品 - 0#车用柴油 （VI）', '', '', '', '', '', 'RMB', '',
    # 635065.12, 635065.12, 0.0, 'Manager', '', 'Manager', '', '', '', '100.654', '6,309.39'],
    # ['10', '*', '', '', '', '', '购货', '22210101', '应交税费 - 应交增值税 - 进项税额', '', '', '', '', '', 'RMB', '',
    # 108535.22, 108535.22, 0.0, 'Manager', '', 'Manager', '', '', '', '', ''],
    # ['10', '*', '', '', '', '', '购货', '220240', '应付账款 - 四川中油九洲北斗科技能源有限公司', '', '', '', '', '', 'RMB',
    # '', 943421.58, 0.0, 943421.58, 'Manager', '', 'Manager', '', '', '', '', '']]
    result = []
    oil_list = []
    com_list = []
    if '购' in original_data[0][6]:
        flag = 0    # 0:购 1:销
    else:
        flag = 1
    for record in original_data:
        if any(key in record[8] for key in oil_key) and '公司' not in record[8]:
#             for r in record[8].replace(' ', '').split('-'):
            for r in record[8].replace('--', '').replace('-', '').split(' '):
                if any(key in r for key in oil_key):
                    oil_list.append(r)
        elif '公司' in record[8]:
#             for r in record[8].replace(' ', '').split('-'):
            for r in record[8].replace('--', '').replace('-', '').split(' '):
                if '公司' in r:
                    com_list.append(r)
    oil_num = len(oil_list)
    com_num = len(com_list)
#     print('oil_num:',oil_num)
#     print('com_num:',com_num)
    if oil_num == 1 and com_num == 1:   # 油品公司一一对应，最简单的匹配
        if flag == 0:
            xf = name_filter(com_list[0])
            gf = name_filter(com)
        else:
            print('此记录为卖出记录，不予统计')
            return 0
            # xf = com
            # gf = com_list[0]
        date = original_data[0][4]
        spmc = oil_list[0]
        Je = original_data[0][16]
        Sl = original_data[0][25] if original_data[0][25] else None
        Dj = original_data[0][26] if original_data[0][26] else None
        if ',' in str(Dj):
            Dj = Dj.replace(',', '')
        result.append((xf, gf, date, spmc, Je, Sl, Dj, path))
    elif oil_num > 1 and com_num == 1:  # 一个公司多个油品：
        if flag == 0:
            # xf = com_list[0]
            # gf = com
            xf = name_filter(com_list[0])
            gf = name_filter(com)
        else:
            print('此记录为卖出记录，不予统计')
            return 0
            # xf = com
            # gf = com_list[0]
        date = original_data[0][4]
        for m in oil_list:
            for n in original_data:
                if m in n[8]:
                    spmc = oil_list[0]
                    Je = n[16]
                    Sl = n[25] if n[25] else None
                    Dj = n[26] if n[26] else None
                    if ',' in str(Dj):
                        Dj = Dj.replace(',', '')
                    result.append((xf, gf, date, spmc, Je, Sl, Dj, path))                    
    elif oil_num > 1 and com_num > 1 and oil_num == com_num: #数量相同一一对应
#         print('oil_num:',oil_num,oil_list)
#         print('com_num:',com_num,com_list)
#         exchange_dict = dict(zip(com_list,oil_list))
        if flag == 0:
#             xf = name_filter(com_list[0])
            gf = name_filter(com)
        else:
            print('此记录为卖出记录，不予统计')
            return 0
        date = original_data[0][4]
        price = []        
        for n in original_data: 
# original_data: [['1', '*', '', '', '2019-01-31', '通用-1', '购货', '140506', '库存商品 - -10号车用柴油（VI）', '', '', '', '', '', \
# 'RMB', '', 982758.62, 982758.62, 0.0, 'Manager', '', 'Manager', '', '', '', '200.00', '4,913.79', 2],
# ['1', '*', '', '', '2019-01-31', '通用-1', '购货', '140506', '应付账款 - 四川中油九洲北斗科技能源有限公司', '', '', '', '', '', \
# 'RMB', '', 982758.62, 982758.62, 0.0, 'Manager', '', 'Manager', '', '', '', '', '', 2]]
            if len(str(n[25])) > 2:
                Je = n[16]
                Sl = n[25] if n[25] else None
                Dj = n[26] if n[26] else None
                if ',' in str(Dj):
                    Dj = Dj.replace(',', '')
                price.append([Je, Sl, Dj, path])
        print(len(price))
        title = []
        for i in range(len(com_list)):
            xf = com_list[i]
            spmc = oil_list[i]
            t = [xf, gf, date, spmc]
            title.append(t)
        print(len(title))
        print(title)
        for i in range(len(price)):
            row = title[i]+ price[i]           
            result.append(tuple(row))
        print('多对多：', result)            
        
    else:   # 多个油品对应多家公司，油品数目与公司数目可能不等
        print('oil_num:',oil_num,oil_list)
        print('com_num:',com_num,com_list)
        print('合并记账，暂时没想到好方法,可先将此文件做记录，后续处理')
        return -1
    return result

def main():
    """
        序时账表头
        ['0期间', '1过账', '2审核', '3作废', '4日期', '5凭证字号', '6摘要', '7科目代码', '8科目名称',
        '9往来单位', '10部门', '11职员', '12自定义项目', '13业务编号', '14币别', '15汇率', '16原币金额',
        '17借方金额', '18贷方金额', '19制单人', '20审核人', '21过账人', '22结算方式', '23结算号',
         '24结算日期', '25数量', '26单价', '27附件数']
        """
    path_list = gci('XSZ')
    error_list = []
    update_list = []
    match_error_list = []
    for k, v in path_list.items():
        for file_path in v:
            final_data = []
            if '序时' not in file_path:
                continue
            print(file_path)
            file_data = read_xls(file_path)

            if file_data:
                # 跳过无关公司的序时账
                if len(file_data[0]) < 27:
                    continue
                if '2019' in str(file_data[0][4]) or '2020' in str(file_data[0][4]):
                    temp = []
                    temp_data = []
                    for r in file_data:
                        if temp and r[4]:  # 判断是否是新账目(按日期）
                            if any(key in temp[0][6] for key in trade_key):  # 判断是否是购销
                                if any(key in temp[0][8] for key in oil_key) and '公司' not in temp[0][8]:  # 是否包含油品
                                    temp_data.append(temp)
                            temp = []
                        temp.append(r)
                    for r in temp_data:
                        try:
                            middle_data = data_merge(k, r, file_path)
                            if middle_data == -1:
                                match_error_list.append(file_path)  # 添加没想到好方法文件的路径信息(合并记账）
                                continue
                            elif middle_data == 0: # 该记录为卖出记录
                                continue
                            if middle_data:
                                for j in middle_data:
                                    final_data.append(j)
                        except:
                            print('有问题公司：', file_path)
                            error_list.append(file_path)  # 添加解析文件出错的路径信息
            # print(final_data)
            update_list.extend(final_data)
    # print('err:',error_list)
    # print('update:',update_list)

    # 上传到数据库
    # 交易表入库sql
    trade_sql = "INSERT INTO financial_exchange (Xf_company_name,Gf_company_name,exchange_date,exchange_good,Je,Sl,Dj,source" \
                ") VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.executemany(trade_sql, update_list)
    connection.commit()
    no_match_list = {}
    for com in match_error_list:
        if com not in no_match_list.keys():
            no_match_list[com]=match_error_list.count(com)

    print('合并记账企业：\n', no_match_list)


def test(k,file_path):
    file_data = read_xls(file_path)
    final_data =[]
    error_data = []
#     print(file_data)
    if file_data:
        # 跳过无关公司的序时账
        if len(file_data[0]) < 27:
            print('格式有误')
        if '2019' in str(file_data[0][4]) or '2020' in str(file_data[0][4]):
            temp = []
            temp_data = []
            index = 1            
            for r in file_data:
                index += 1
                r = r + [index,]
                if temp and r[4]:  # 判断是否是新账目
                    if any(key in temp[0][6] for key in trade_key):  # 判断是否是购销
                        if any(key in temp[0][8] for key in oil_key) and '公司' not in temp[0][8]:  # 是否包含油品
                            temp_data.append(temp)
                    temp = []
                temp.append(r)
#             print(temp_data)
            for r in temp_data:
                try:
                    middle_data = data_merge(k, r, file_path)
                    if middle_data == -1:
                        # print('合并记账')
                        error_data.append(r)
                        continue
                    if middle_data:
                        for j in middle_data:
                            final_data.append(j)
                except Exception as e:
                    print(str(e))
                    print('有问题公司：', file_path)
#     print(final_data)
    return error_data


if __name__ == "__main__":
    # main()
    err = test('中油银海（大连）石油化工有限公司','XSZ\\2020-03-05\\中油银海（大连）石油化工有限公司\\0ef2db7d-ccd6-459c-a4f5-8377659fea20_中油银海（大连）石油化工有限公司序时账.xls')
    

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
oil_num: 8 ['10号车用柴油（VI）', '10号车用柴油（VI）', '10号车用柴油（VI）', '10号车用柴油（VI）', '0号车用柴油（VI）', '10号车用柴油（VI）', '0号车用柴油（VI）', '10号车用柴油（VI）']
com_num: 8 ['江苏金翔石油化工有限公司', '江苏金翔石油化工有限公司', '江苏金翔石油化工有限公司', '中国石化销售有限公司山东菏泽石油分公司', '中国石油天然气股份有限公司贵州销售分公司', '利津县瑞辰化工有限公司', '中海油华东销售有限公司', '中海油华东销售有限公司']
8
8
[['江苏金翔石油化工有限公司', '中油银海（大连）石油化工有限公司', '2019-01-31', '10号车用柴油（VI）'], ['江苏金翔石油化工有限公司', '中油银海（大连）石油化工有限公司', '2019-01-31', '10号车用柴油（VI）'], ['江苏金翔石油化工有限公司', '中油银海（大连）石油化工有限公司', '2019-01-31', '10号车用柴油（VI）'], ['中国石化销售有限公司山东菏泽石油分公司', '中油银海（大连）石油化工有限公司', '2019-01-31', '10号车用柴油（VI）'], ['中国石油天然气股份有限公司贵州销售分公司', '中油银海（大连）石油化工有限公司', '2019-01-31', '0号车用柴油（VI）'], ['利津县瑞辰化工有限公司', '中油银海（大连）石油化工有限公司', '2019-01-31', '10号车用柴油（VI）'], ['中海油华东销售有限公司', '中油银海（大连）石油化工有限公司', '2019-01-31', '0号车用柴油（VI）'], ['中海油华东销售有限公司', '中油银海（大连）石油化工有限公司', '2019-01-31', '10号车用柴油（VI）']]
多对多： [('江苏金翔石油化工有限公司', '中油银海（大连）石油化工有限公司', '2019-01-31', '10号车用柴油（VI）', 982758.62, '200.00', '

In [96]:
print(err[2])
for i in err[2]:
    print(i[-1])

[['11', '*', '', '', '2019-11-30', '通用-1', '购货', '140519', '库存商品 - 车用柴油0#VI', '', '', '', '', '', 'RMB', '', 375847.95, 375847.95, 0.0, 'Manager', '', 'Manager', '', '', '', '62.11', '6,051.33', 1678], ['11', '*', '', '', '', '', '购货', '22210101', '应交税费 - 应交增值税 - 进项税额', '', '', '', '', '', 'RMB', '', 48860.23, 48860.23, 0.0, 'Manager', '', 'Manager', '', '', '', '', '', 1679], ['11', '*', '', '', '', '', '购货', '220230', '应付账款 - 浙江舟山鼎源石油化工有限公司', '', '', '', '', '', 'RMB', '', 424708.18, 0.0, 424708.18, 'Manager', '', 'Manager', '', '', '', '', '', 1680], ['11', '*', '', '', '', '', '购货', '140504', '库存商品 - 0号车用柴油（VI）', '', '', '', '', '', 'RMB', '', 115044.25, 115044.25, 0.0, 'Manager', '', 'Manager', '', '', '', '20.00', '5,752.21', 1681], ['11', '*', '', '', '', '', '购货', '22210101', '应交税费 - 应交增值税 - 进项税额', '', '', '', '', '', 'RMB', '', 14955.75, 14955.75, 0.0, 'Manager', '', 'Manager', '', '', '', '', '', 1682], ['11', '*', '', '', '', '', '购货', '220235', '应付账款 - 江苏省石油集团镇江金山油料供应有限责任公司

In [48]:
file_path = 'XSZ\\2020-03-05\\中油银海（大连）石油化工有限公司\\0ef2db7d-ccd6-459c-a4f5-8377659fea20_中油银海（大连）石油化工有限公司序时账.xls'

file_data = read_xls(file_path)
final_data =[]
# print(file_data)
if file_data:
    # 跳过无关公司的序时账
    if len(file_data[0]) < 27:
        print('格式有误')
    if '2019' in str(file_data[0][4]) or '2020' in str(file_data[0][4]):
        temp = []
        temp_data = []
        index = 1
        for r in file_data:
#             print(temp)
            index += 1
            r = r + [index,]
            if temp and r[4]: # 判断是否是新账目
                if any(key in temp[0][6] for key in trade_key):  # 判断是否是购销
                    if any(key in temp[0][8] for key in oil_key) and '公司' not in temp[0][8]:  # 是否包含油品
                        temp_data.append(temp)
                temp = []
#             r = [index,] + r
            temp.append(r)
        print(temp_data)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
[[['1', '*', '', '', '2019-01-31', '通用-1', '购货', '140506', '库存商品 - -10号车用柴油（VI）', '', '', '', '', '', 'RMB', '', 982758.62, 982758.62, 0.0, 'Manager', '', 'Manager', '', '', '', '200.00', '4,913.79', 2], ['1', '*', '', '', '', '', '购货', '22210101', '应交税费 - 应交增值税 - 进项税额', '', '', '', '', '', 'RMB', '', 157241.38, 157241.38, 0.0, 'Manager', '', 'Manager', '', '', '', '', '', 3], ['1', '*', '', '', '', '', '购货', '220223', '应付账款 - 江苏金翔石油化工有限公司', '', '', '', '', '', 'RMB', '', 1140000.0, 0.0, 1140000.0, 'Manager', '', 'Manager', '', '', '', '', '', 4], ['1', '*', '', '', '', '', '购货', '140506', '库存商品 - -10号车用柴油（VI）', '', '', '', '', '', 'RMB', '', 723275.86, 723275.86, 0.0, 'Manager', '', 'Manager', '', '', '', '150.00', '4,821.84', 5], ['1', '*', '', '', '', '', '购货', '22210101', '应交税费 - 应交增值税 - 进项税额', '', '', '', '', '', 'RMB', '', 115724.14, 115724.14, 0.0, 'Manager', '', 'Manager', '', '', '', '', '', 6], ['1', '*'

In [61]:
print(temp_data[0])

[['1', '*', '', '', '2019-01-31', '通用-1', '购货', '140506', '库存商品 - -10号车用柴油（VI）', '', '', '', '', '', 'RMB', '', 982758.62, 982758.62, 0.0, 'Manager', '', 'Manager', '', '', '', '200.00', '4,913.79', 2], ['1', '*', '', '', '', '', '购货', '22210101', '应交税费 - 应交增值税 - 进项税额', '', '', '', '', '', 'RMB', '', 157241.38, 157241.38, 0.0, 'Manager', '', 'Manager', '', '', '', '', '', 3], ['1', '*', '', '', '', '', '购货', '220223', '应付账款 - 江苏金翔石油化工有限公司', '', '', '', '', '', 'RMB', '', 1140000.0, 0.0, 1140000.0, 'Manager', '', 'Manager', '', '', '', '', '', 4], ['1', '*', '', '', '', '', '购货', '140506', '库存商品 - -10号车用柴油（VI）', '', '', '', '', '', 'RMB', '', 723275.86, 723275.86, 0.0, 'Manager', '', 'Manager', '', '', '', '150.00', '4,821.84', 5], ['1', '*', '', '', '', '', '购货', '22210101', '应交税费 - 应交增值税 - 进项税额', '', '', '', '', '', 'RMB', '', 115724.14, 115724.14, 0.0, 'Manager', '', 'Manager', '', '', '', '', '', 6], ['1', '*', '', '', '', '', '购货', '220223', '应付账款 - 江苏金翔石油化工有限公司', '', '', '', '', '

In [37]:
file_data = read_xls(file_path)
print(file_data)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
[['1', '*', '', '', '2019-01-31', '通用-1', '购货', '140506', '库存商品 - -10号车用柴油（VI）', '', '', '', '', '', 'RMB', '', 982758.62, 982758.62, 0.0, 'Manager', '', 'Manager', '', '', '', '200.00', '4,913.79'], ['1', '*', '', '', '', '', '购货', '22210101', '应交税费 - 应交增值税 - 进项税额', '', '', '', '', '', 'RMB', '', 157241.38, 157241.38, 0.0, 'Manager', '', 'Manager', '', '', '', '', ''], ['1', '*', '', '', '', '', '购货', '220223', '应付账款 - 江苏金翔石油化工有限公司', '', '', '', '', '', 'RMB', '', 1140000.0, 0.0, 1140000.0, 'Manager', '', 'Manager', '', '', '', '', ''], ['1', '*', '', '', '', '', '购货', '140506', '库存商品 - -10号车用柴油（VI）', '', '', '', '', '', 'RMB', '', 723275.86, 723275.86, 0.0, 'Manager', '', 'Manager', '', '', '', '150.00', '4,821.84'], ['1', '*', '', '', '', '', '购货', '22210101', '应交税费 - 应交增值税 - 进项税额', '', '', '', '', '', 'RMB', '', 115724.14, 115724.14, 0.0, 'Manager', '', 'Manager', '', '', '', '', ''], ['1', '*', '', '', '', ''

In [43]:
print(file_data[0])

['1', '*', '', '', '2019-01-31', '通用-1', '购货', '140506', '库存商品 - -10号车用柴油（VI）', '', '', '', '', '', 'RMB', '', 982758.62, 982758.62, 0.0, 'Manager', '', 'Manager', '', '', '', '200.00', '4,913.79']


In [100]:
t = '应付账款 - 准格尔旗富华加油站'
a = t.replace('--', '').replace('-', '').split(' ')
if any(key in a for key in trade_key):
    print('a')

In [89]:
test1 = [['1','2'],['4','5']]
test2 = [['1','2'],['4','5']]
test1+test2

[['1', '2'], ['4', '5'], ['1', '2'], ['4', '5']]

In [75]:
# str_test = '库存商品 - -10号车用柴油（VI）'
str_test = '应付账款 - 江苏金翔石油化工有限公司'
str_test.replace('--','').replace('-','').split('  ')


['应付账款', '江苏金翔石油化工有限公司']

In [16]:
list_test = [('江苏金翔石油化工有限公司', '中油银海（大连）石油化工有限公司', '2019-04-30', '0号车用柴油（VI)0号车用柴油（VI)', 1682291.1, '274.90', '6119.65', 'XSZ\\2020-03-05\\中油银海（大连）石油化工有限公司\\0ef2db7d-ccd6-459c-a4f5-8377659fea20_中油银海（大连）石油化工有限公司序时账.xls'), \
             ('中海油华东销售有限公司', '中油银海（大连）石油化工有限公司', '2019-04-30', '0号车用柴油（VI）', 2177075.07, '400.00', '5442.69', 'XSZ\\2020-03-05\\中油银海（大连）石油化工有限公司\\0ef2db7d-ccd6-459c-a4f5-8377659fea20_中油银海（大连）石油化工有限公司序时账.xls'), \
             ('中国石油天然气股份有限公司云南临沧销售分公司', '中油银海（大连）石油化工有限公司', '2019-04-30', '0号车用柴油（VI）', 1122123.89, '200.00', '5610.62', 'XSZ\\2020-03-05\\中油银海（大连）石油化工有限公司\\0ef2db7d-ccd6-459c-a4f5-8377659fea20_中油银海（大连）石油化工有限公司序时账.xls')]


D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Dj' at row 1")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Dj' at row 2")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Dj' at row 3")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Dj' at row 4")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 5")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Dj' at row 5")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data tru

D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 29")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 40")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 46")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 48")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 54")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Data truncated for column 'Sl' at row 55")
  result = self._query(query)
D:\Selvaria\Anaconda\lib\site-packages\pymysql\cursors.py:165: Warning: (1265, "Da